# Ensemble Analysis

This notebook reads in and formats outputs from ensemble simulations so that you can evaluate and plot

### Import packages

In [35]:
#import packages
import flopy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mp
import pandas as pd
import numpy as np
import os


#additional analysis tools
import flopy.utils.binaryfile as bf
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


np.set_printoptions(threshold=np.inf)

#jupyter specific--included to show plots in notebook
%matplotlib inline 

# Setup static variables

In [36]:
nrow = 50 #number of rows
ncol = 50 #number of columns
nlay = 3  #number of layers
dx= 1000 #width of grid cells in x direction 
dy= 1000 #width of grid cells in y direction 
Lx = ncol*dx  #width of domain in x
Ly = nrow*dy #width of domain in y

ensembles = [[2,2,2,2,2,2,2],
[2,2,2,1,2,2,2], 
[2,2,2,3,2,2,2],
[2,2,2,2,2,2,1],
[2,2,2,2,2,2,3],
[2,2,2,2,2,1,2],
[2,2,2,2,2,3,2],
[2,2,2,1,3,3,1],
[2,2,2,3,1,1,3],
[3,2,2,2,2,2,3],
[1,2,2,2,2,2,1]]

n_ens = len(ensembles)
print(n_ens, 'Ensembles to evaluate')


11 Ensembles to evaluate


## Read in the heads and create timeseries at points

In [37]:
#setup well locations and observation points
# To pull outwant the row of observations starting at well location and edge of riparian community @ horizontal point!!!!
en_list = [2222222,
2221222, 
2223222,
2222221,
2222223,
2222212,
2222232,
2221331,
2223113,
3222223,
1222221]

welli_loc = (0,12, 14) #ag irrigation well 
river_head = (0,23, 14) # river point
farm_loc = (0,20,18) # corner nearest to the farm!

#setup timeseries
final_array = np.zeros(shape = (11,5))

print(final_array)
ag_head_ts = []  # timeseries of head at agg well
ag_farm_head = [] # timeseries at the farm location
riv_heads_ts = []
# Loop through the ensembles and read the data
root_name = 'ensemble_' #root of the run names
for i in range(len(ensembles)):
    name = root_name  #Create the file name from list 
    for k in range(len(ensembles[i])):
        string = str(ensembles[i][k])
        name = name + string   

    #read in the head and water budget files
    headobj = flopy.utils.binaryfile.HeadFile(name+'.hds')
   
    #Extract out time series of heads at points of interest
    ag_head_ts.append(np.mean(headobj.get_ts(welli_loc)))
    riv_heads_ts.append(np.mean(headobj.get_ts(river_head)))
    ag_farm_head.append(np.mean(headobj.get_ts(farm_loc)))
    times = headobj.get_times()
    #print(budgobj.get_unique_record_names())

    #print(riv_heads_ts)
print(len(en_list))


#en_list.append(riv_heads_ts)
#print(en_list)

print(riv_heads_ts)
print(ag_head_ts)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
11
[321.0797, 320.94788, 321.20828, 318.2498, 321.19196, 321.09705, 321.06247, 315.04498, 321.26138, 321.16586, 321.7627]
[319.70474, 319.20267, 320.19226, 316.9606, 319.91058, 319.72537, 319.6843, 313.68924, 320.35843, 320.13367, 318.6719]


In [38]:
# Calculate Slope and Graph
# Can plot into excel
# variable.write() --> write out into python
slope_array = np.subtract(riv_heads_ts,ag_head_ts)/ ((23-12)* dx) # slopes from ag well to river
print(slope_array)


slopes2 = np.subtract(riv_heads_ts,ag_farm_head)/ ((23-20)* dx) # slopes from farm to river
print(slopes2)




[1.2499723e-04 1.5865534e-04 9.2365612e-05 1.1719860e-04 1.1648837e-04
 1.2469760e-04 1.2528853e-04 1.2324940e-04 8.2086735e-05 9.3836003e-05
 2.8098089e-04]
[0.0005449  0.00056474 0.00052548 0.00026545 0.00057383 0.00054305
 0.00054671 0.00019347 0.00054385 0.00058587 0.00048851]


## Read in the river leakage

In [40]:
#setup arrays to store the values
leakage_array = np.zeros((len(ensembles), len(times), 49))

# Loop through the ensembles and read the data
root_name = 'ensemble_' #root of the run names
for i in range(len(ensembles)):
    name = root_name  #Create the file name from list 
    for k in range(len(ensembles[i])):
        string = str(ensembles[i][k])
        name = name + string   

    #water budget files
    budgobj = flopy.utils.binaryfile.CellBudgetFile(name+'.cbc')   

    rowcounter = -1
    for t in times:
        rowcounter += 1

        templeak = budgobj.get_data(text='RIVER LEAKAGE', totim=t) # 
        leakage_array[i, rowcounter,:] = templeak[0].q

leakage_all = np.sum(leakage_array, axis = (1,2))/61 # sum up across entire time series across all cells ( divide by total time steps to get avg river leakage)


# Add All Variables to the final_array so it plots nicely
for l in range(len(ensembles)):
   final_array[l,0]= en_list[l]
   final_array[l,1] = riv_heads_ts[l]
   final_array[l,2] = slope_array[l]
   final_array[l,3] = slopes2[l]
   final_array[l,4] = leakage_all[l]

# Write out CSV
pd.DataFrame(final_array).to_csv('metrics.csv')
#np.savetxt('test.csv', en_list, delimiter=",")

# Ways to Plot
# 1. Difference between Scenarios ( Average across all params, all results plotted by scenario ( 4 colors) )
# 2. Param space sensitivity ( Pick on scenario and plot)